In [ ]:
import h5py
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
insat_data = h5py.File('3DIMG_01APR2022_0600_L1B_STD.h5', 'r')

In [ ]:
# data.close()

In [ ]:
for item in list(insat_data.keys()):
    print(item, np.array(insat_data[item]).shape)

In [ ]:
# fig = px.imshow(
#     np.array(insat_data['IMG_MIR'][0])
# )
# fig.update_layout(height=750, width=750)
# fig

In [ ]:
print(f"IMG_MIR shape: {np.array(insat_data['IMG_MIR']).shape}")
print(f"Latitude shape: {np.array(insat_data['Latitude']).shape}")
print(f"Longitude shape: {np.array(insat_data['Longitude']).shape}")

In [ ]:
np.array(insat_data['IMG_MIR'])

In [ ]:
np.deg2rad(np.array(insat_data['Latitude_WV']))
def get_cartesian(lat=None,lon=None):
    lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    print(lat.shape, lon.shape)
    R = 6371 # radius of the earth
    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    z = R *np.sin(lat)
    return x[::10000],y[::10000],z[::10000]

In [ ]:
np.deg2rad(np.array(insat_data['Latitude_WV']))

In [ ]:
import numpy as np
v = np.array(insat_data['IMG_MIR']).flatten()
normalized_v = v / np.sqrt(np.sum(v**2))
normalized_v *= 1000000
normalized_v


In [ ]:
# %pylab 
# %matplotlib inline
# plt.hist(normalized_v)

In [ ]:
x,y,z = get_cartesian(np.array(insat_data['Latitude']).flatten(), np.array(insat_data['Longitude']).flatten())

# fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
#                                    marker=dict(
#                                         color='LightSkyBlue',
#                                         size=1,
# #                                         line=dict(
# #                                             color='MediumPurple',
# #                                             width=2
# #                                         )
#                                     ),
#                                    mode='markers')])
# # fig.update_layout(height=750, width=750)
# fig.show()

In [ ]:
x.shape

In [ ]:
np.square(2)

In [ ]:
norm_d = normalized_v[::10000]
xd,yd,zd = [],[],[]
for xc,yc,zc, norm, in zip(x,y,z, norm_d):
    den = np.sqrt(xc**2+yc**2+zc**2)
    tmp_x = (np.abs(xc)+norm*(np.abs(xc)/den)) * np.sign(xc)
    tmp_y = (np.abs(yc)+norm*(np.abs(yc)/den)) * np.sign(yc)
    tmp_z = (np.abs(zc)+norm*(np.abs(zc)/den)) * np.sign(zc)
#     print(tmp_x,)
    xd.append(tmp_x)
    yd.append(tmp_y)
    zd.append(tmp_z)
# xc,yd,zd = [np.array(item) for item in zip(zd,yd,zd)]

In [ ]:
np.array(xd).shape

In [ ]:
# fig = go.Figure()

# # Add traces
# # fig.add_trace(go.Scatter(x=x, y=y,
# #                     mode='markers',
# #                     name='markers'))
# fig.add_trace(go.Scatter(x=xd, y=yd,
#                     mode='markers',
#                     name='markers'))
# fig

In [ ]:
import urllib.request, json
# url = "http://maps.googleapis.com/maps/api/geocode/json?address=google"
url = "https://raw.githubusercontent.com/Nov05/playground-fireball/master/data/earth.json"
response = urllib.request.urlopen(url)
json_data = json.loads(response.read())

In [ ]:
# 纬线 Parallels
# x0 = json_data['data'][0]['x']
# y0 = json_data['data'][0]['y']
# z0 = json_data['data'][0]['z']
# len(x0)
# 经线 Meridians
# x1 = json_data['data'][1]['x']
# y1 = json_data['data'][1]['y']
# z1 = json_data['data'][1]['z']
# len(x1)
# # blue base (dots)
# x2 = json_data['data'][2]['x']
# y2 = json_data['data'][2]['y']
# z2 = json_data['data'][2]['z']
# len(x2)
# # white continents (lines)
x3 = json_data['data'][3]['x']
y3 = json_data['data'][3]['y']
z3 = json_data['data'][3]['z']
# len(x3)

In [ ]:
import matplotlib.cm as cm
import matplotlib as mpl
import matplotlib.colors as clr
def transition_colors(length=10, name="Reds"):
    """
    Create colorscale for convergence plots, returns a list of colors.
    Parameters
    ----------
    length : int
        The length of the colorscale.
    name : string, default: 'jet', optional
        Name of the colorscale.
    Returns
    -------
    colors: list
    """
    cmap = mpl.cm.get_cmap(name, length)
    colors = []
    for i in range(cmap.N):
        rgb = cmap(i)[:3]
        colors.append(mpl.colors.rgb2hex(rgb))
    return colors
transition_colors(length=10)

In [ ]:
traces = []
for xc,yc,zc, xcd, ycd, zcd, norm in zip(x,y,z,xd,yd,zd, norm_d):
    tmp_trace = go.Scatter3d(
    x=[xc,xcd],
    y=[yc,ycd],
    z=[zc,zcd],
    mode='lines',
    marker=dict(
        size=10,
        color=norm,  
        colorscale='Viridis',
        showscale=False,
#         line=dict(width=1, color=color_land)
        ),

    showlegend=False,
    name='land'
    )
    traces.append(tmp_trace)

In [ ]:
len(traces)

In [ ]:
color_background = 'black'
color_lines = 'darkblue' # Parallels and Meridians
color_base = 'darkblue'
color_land = 'whitesmoke'
opacity_base = 0.1
opacity_land = 0.8
size_lines = 1

# Parallels 
# trace0 = go.Scatter3d(
#     x=x0,
#     y=y0,
#     z=z0,
#     mode='lines',
#     marker=dict(
#         size=size_lines,
#         color=color_lines, # set color to an array/list of desired values 
#         opacity=opacity_base,
#         showscale=False,
#         line=dict(
#             width=size_lines, 
#             color=color_base)
#         ),
#     showlegend=False,
#     name='Parallel'
#     )
# Meridians
trace1 = go.Scatter3d(
    x=x1,
    y=y1,
    z=z1,
    mode='lines',
    marker=dict(
        size=size_lines,
        color=color_lines,  
        opacity=opacity_base,
        showscale=False,
        line=dict(
            width=size_lines, 
            color=color_base,)
        ),
    showlegend=False,
    name='Meridian'
    )
# # base surface (dots)
# trace2 = go.Scatter3d(
#     x=x2,
#     y=y2,
#     z=z2,
#     mode='markers',
#     marker=dict(
#         size=2,
#         color=color_base,  
#         opacity=opacity_base,
#         showscale=False,
#         line=dict(width=1, color=color_base)
#         ),
#     showlegend=False,
#     name='by Wenjing Liu',
#     )
# land
trace3 = go.Scatter3d(
    x=x3,
    y=y3,
    z=z3,
    mode='lines',
    marker=dict(
        size=1,
        color=color_land,  
        opacity=opacity_land,
        showscale=False,
        line=dict(width=1, color=color_land)
        ),
    showlegend=False,
    name='land'
    )

trace4 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=1,
        color=color_land,  
        opacity=opacity_land,
        showscale=False,
#         line=dict(width=1, color=color_land)
        ),
    showlegend=False,
    name='land'
    )

# trace5 = go.Scatter3d(
#     x=x,
#     y=y,
#     z=z,
#     mode='markers',
#     marker=dict(
#         size=1,
# #         color=color_land,  
# #         opacity=opacity_land,
#         showscale=False,
# #         line=dict(width=1, color=color_land)
#         ),
#     showlegend=False,
#     name='land'
#     )
data = [
#     trace0, 
    trace1, 
#     trace2, 
    trace3,
    trace4
]

data.extend(traces)

layout = go.Layout(
    autosize=False,
    width=1300,
    height=800,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    title=dict(
        text='title',
        x=0.015, y=0.95),
    scene=dict(
        xaxis=dict(
            title="x axis",
#             color=color_background,
#             backgroundcolor=color_background,
            showaxeslabels=False,
            showline=False,
            showgrid=False,
            zeroline=False,
            ),
        yaxis=dict(
            title="y axis",
#             color=color_background,
#             backgroundcolor=color_background,
            showaxeslabels=False,
            showline=False,
            showgrid=False,
            zeroline=False,
            ),
        zaxis=dict(
            title="z axis",
#             color=color_background, 
#             backgroundcolor=color_background,
            showaxeslabels=False,
            showline=False,
            showgrid=False,
            zeroline=False,
            ),
        ),
#     paper_bgcolor=color_background,
#     plot_bgcolor=color_background,
)
fig = go.Figure(data=data, layout=layout)
# fig.show()
fig.write_html('mir.html')

In [ ]:
9